In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier

In [2]:
df = pd.read_csv("1805719.csv")
df.head()

,business_code,cust_number,name_customer,clear_date,buisness_year,doc_id,posting_date,document_create_date,document_create_date.1,due_in_date,invoice_currency,document type,posting_id,area_business,total_open_amount,baseline_create_date,cust_payment_terms,invoice_id,isOpen
0,U001,0200769623,WAL-MAR in,2019-04-02 00:00:00,2019.0,1.929010e+09,2019-03-22,20190322,20190322,20190406.0,USD,RV,1.0,NaN,2684.80,20190322.0,NAH4,1.929010e+09,0
1,CA02,100034823,ZARCO associates,2019-03-26 00:00:00,2019.0,2.960535e+09,2019-03-14,20190314,20190314,20190325.0,CAD,RV,1.0,NaN,2638.89,20190315.0,CA10,2.960535e+09,0
2,U001,200794332,COST associates,2019-07-25 00:00:00,2019.0,1.929607e+09,2019-07-10,20190710,20190710,20190725.0,USD,RV,1.0,NaN,43447.57,20190710.0,NAAX,1.929607e+09,0
3,U001,0200707741,SMART & co,2020-02-25 00:00:00,2020.0,1.930482e+09,2020-02-06,20200205,20200206,20200221.0,USD,RV,1.0,NaN,118601.49,20200206.0,NAA8,1.930482e+09,0
4,CA02,0140105686,SYSC corp,2019-02-28 00:00:00,2019.0,2.960528e+09,2019-02-13,20190213,20190213,20190224.0,CAD,RV,1.0,NaN,40308.94,20190214.0,CA10,2.960528e+09,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   business_code           50000 non-null  object 
 1   cust_number             50000 non-null  object 
 2   name_customer           50000 non-null  object 
 3   clear_date              45632 non-null  object 
 4   buisness_year           50000 non-null  float64
 5   doc_id                  50000 non-null  float64
 6   posting_date            50000 non-null  object 
 7   document_create_date    50000 non-null  int64  
 8   document_create_date.1  50000 non-null  int64  
 9   due_in_date             50000 non-null  float64
 10  invoice_currency        50000 non-null  object 
 11  document type           50000 non-null  object 
 12  posting_id              50000 non-null  float64
 13  area_business           0 non-null      float64
 14  total_open_amount       50000 non-null

In [6]:
df['document_create_date']= pd.to_datetime(df.document_create_date, format = '%Y%m%d') 
df['document_create_date.1']= pd.to_datetime(df['document_create_date.1'], format = '%Y%m%d')
df['due_in_date']= pd.to_datetime(df.due_in_date, format = '%Y%m%d') 
df['baseline_create_date']= pd.to_datetime(df.baseline_create_date, format = '%Y%m%d')
df['posting_date']= pd.to_datetime(df.posting_date)
df['clear_date']= pd.to_datetime(df.clear_date).dt.normalize()

df = df.rename(columns={'document type': 'document_type'})

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   business_code           50000 non-null  object        
 1   cust_number             50000 non-null  object        
 2   name_customer           50000 non-null  object        
 3   clear_date              45632 non-null  datetime64[ns]
 4   buisness_year           50000 non-null  float64       
 5   doc_id                  50000 non-null  float64       
 6   posting_date            50000 non-null  datetime64[ns]
 7   document_create_date    50000 non-null  datetime64[ns]
 8   document_create_date.1  50000 non-null  datetime64[ns]
 9   due_in_date             50000 non-null  datetime64[ns]
 10  invoice_currency        50000 non-null  object        
 11  document_type           50000 non-null  object        
 12  posting_id              50000 non-null  float6

In [8]:
df.isnull().sum()

business_code                 0
cust_number                   0
name_customer                 0
clear_date                 4368
buisness_year                 0
doc_id                        0
posting_date                  0
document_create_date          0
document_create_date.1        0
due_in_date                   0
invoice_currency              0
document_type                 0
posting_id                    0
area_business             50000
total_open_amount             0
baseline_create_date          0
cust_payment_terms            0
invoice_id                    5
isOpen                        0
dtype: int64

In [9]:
#dropping area business
df.drop(['area_business'], axis=1,inplace=True)

#dropping doccument create date
df = df.drop(columns=['document_create_date'])

#checking unique values in posting id
print(df.posting_id.value_counts())

#since there is only 1 value hence it is a constant column, hence droping
df = df.drop(columns=['posting_id'])


1.0    50000
Name: posting_id, dtype: int64


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   business_code           50000 non-null  object        
 1   cust_number             50000 non-null  object        
 2   name_customer           50000 non-null  object        
 3   clear_date              45632 non-null  datetime64[ns]
 4   buisness_year           50000 non-null  float64       
 5   doc_id                  50000 non-null  float64       
 6   posting_date            50000 non-null  datetime64[ns]
 7   document_create_date.1  50000 non-null  datetime64[ns]
 8   due_in_date             50000 non-null  datetime64[ns]
 9   invoice_currency        50000 non-null  object        
 10  document_type           50000 non-null  object        
 11  total_open_amount       50000 non-null  float64       
 12  baseline_create_date    50000 non-null  dateti

In [11]:
#separating dataframe by isopen =1
df1 = df[df['isOpen'] == 0]
isActive = df[df['isOpen'] == 1]

In [12]:
df1.shape

(45632, 16)

In [13]:
print(df1.isOpen.unique())
#since isopwn has only 0 in it, implies it a constant column, hence dropping it
df1 = df1.drop(columns=['isOpen'])

[0]


In [14]:
df1.shape

(45632, 15)

In [15]:
#sorting df according to doc create dt 1
df1 = df1.sort_values(by="document_create_date.1")
df1.head()

,business_code,cust_number,name_customer,clear_date,buisness_year,doc_id,posting_date,document_create_date.1,due_in_date,invoice_currency,document_type,total_open_amount,baseline_create_date,cust_payment_terms,invoice_id
23850,CA02,0140104409,LOB,2019-12-27,2019.0,9.500000e+09,2019-03-29,2018-10-30,2018-11-09,CAD,X2,383.64,2018-10-30,CA10,NaN
14801,CA02,0140104409,LOB corp,2019-12-27,2019.0,9.500000e+09,2019-03-29,2018-12-04,2018-12-14,CAD,X2,305.64,2018-12-04,CA10,NaN
25463,U001,0200726979,BJ'S trust,2019-01-15,2019.0,1.928538e+09,2018-12-30,2018-12-30,2019-01-14,USD,RV,241.59,2018-12-30,NAA8,1.928538e+09
1593,U001,0200769623,WAL-MAR associates,2019-01-09,2019.0,1.928542e+09,2018-12-30,2018-12-30,2019-01-14,USD,RV,38289.77,2018-12-30,NAH4,1.928542e+09
11223,U001,0200769623,WAL-MAR corporation,2019-01-15,2019.0,1.928543e+09,2018-12-30,2018-12-30,2019-01-14,USD,RV,16067.29,2018-12-30,NAH4,1.928543e+09


In [16]:
df1.invoice_id.nunique()

45627

In [17]:
#sincle alonst every id unique hence we dont need it
df1 = df1.drop(columns=['invoice_id'])

In [18]:
df1.shape

(45632, 14)

In [19]:
#since posting date cabt be small than document create date
df1.drop(df1[(df1['document_create_date.1'] <= df1['posting_date']) == False].index, inplace = True) 

In [20]:
df1.shape

(45632, 14)

In [21]:
(df1['document_create_date.1'] <= df1['posting_date']).value_counts()

True    45632
dtype: int64

In [22]:
(df1['document_create_date.1'] <= df1['due_in_date']).value_counts()

True     45527
False      105
dtype: int64

In [23]:
#doc create vs due in date
df1.drop(df1[(df1['document_create_date.1'] <= df1['due_in_date']) == False].index, inplace = True) 

In [24]:
(df1['document_create_date.1'] <= df1['due_in_date']).value_counts()

True    45527
dtype: int64

In [25]:
#creating delay column
df1["delay"] = df1["clear_date"] - df1["due_in_date"]
df1["delay"] = df1["delay"].dt.days.astype('int32')
df1.head()

,business_code,cust_number,name_customer,clear_date,buisness_year,doc_id,posting_date,document_create_date.1,due_in_date,invoice_currency,document_type,total_open_amount,baseline_create_date,cust_payment_terms,delay
23850,CA02,0140104409,LOB,2019-12-27,2019.0,9.500000e+09,2019-03-29,2018-10-30,2018-11-09,CAD,X2,383.64,2018-10-30,CA10,413
14801,CA02,0140104409,LOB corp,2019-12-27,2019.0,9.500000e+09,2019-03-29,2018-12-04,2018-12-14,CAD,X2,305.64,2018-12-04,CA10,378
25463,U001,0200726979,BJ'S trust,2019-01-15,2019.0,1.928538e+09,2018-12-30,2018-12-30,2019-01-14,USD,RV,241.59,2018-12-30,NAA8,1
1593,U001,0200769623,WAL-MAR associates,2019-01-09,2019.0,1.928542e+09,2018-12-30,2018-12-30,2019-01-14,USD,RV,38289.77,2018-12-30,NAH4,-5
11223,U001,0200769623,WAL-MAR corporation,2019-01-15,2019.0,1.928543e+09,2018-12-30,2018-12-30,2019-01-14,USD,RV,16067.29,2018-12-30,NAH4,1


In [26]:
#dropping baseline date 
df1 = df1.drop(columns=['baseline_create_date'])

In [27]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45527 entries, 23850 to 39425
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   business_code           45527 non-null  object        
 1   cust_number             45527 non-null  object        
 2   name_customer           45527 non-null  object        
 3   clear_date              45527 non-null  datetime64[ns]
 4   buisness_year           45527 non-null  float64       
 5   doc_id                  45527 non-null  float64       
 6   posting_date            45527 non-null  datetime64[ns]
 7   document_create_date.1  45527 non-null  datetime64[ns]
 8   due_in_date             45527 non-null  datetime64[ns]
 9   invoice_currency        45527 non-null  object        
 10  document_type           45527 non-null  object        
 11  total_open_amount       45527 non-null  float64       
 12  cust_payment_terms      45527 non-null  ob

In [28]:
#separating delay from rest of dataframe
Y = df1["delay"]
X = df1.drop(columns=["clear_date", "delay"],axis=1)   


In [29]:
print(X.shape)
print(Y.shape)

(45527, 12)
(45527,)


In [30]:
#splitting into test train
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)



In [31]:
#splitting into train val
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, shuffle=False)

In [32]:
X_train.head()

,business_code,cust_number,name_customer,buisness_year,doc_id,posting_date,document_create_date.1,due_in_date,invoice_currency,document_type,total_open_amount,cust_payment_terms
23850,CA02,0140104409,LOB,2019.0,9.500000e+09,2019-03-29,2018-10-30,2018-11-09,CAD,X2,383.64,CA10
14801,CA02,0140104409,LOB corp,2019.0,9.500000e+09,2019-03-29,2018-12-04,2018-12-14,CAD,X2,305.64,CA10
25463,U001,0200726979,BJ'S trust,2019.0,1.928538e+09,2018-12-30,2018-12-30,2019-01-14,USD,RV,241.59,NAA8
1593,U001,0200769623,WAL-MAR associates,2019.0,1.928542e+09,2018-12-30,2018-12-30,2019-01-14,USD,RV,38289.77,NAH4
11223,U001,0200769623,WAL-MAR corporation,2019.0,1.928543e+09,2018-12-30,2018-12-30,2019-01-14,USD,RV,16067.29,NAH4


In [33]:
y_train.head()

23850    413
14801    378
25463      1
1593      -5
11223      1
Name: delay, dtype: int32

In [34]:
#due month train
X_train['due_month'] = X_train['due_in_date'].dt.month
#due week
X_train['due_dayofweek'] = X_train['due_in_date'].dt.dayofweek
#due quater
X_train['due_quarter'] = X_train['due_in_date'].dt.quarter


In [35]:
X_train.head()


,business_code,cust_number,name_customer,buisness_year,doc_id,posting_date,document_create_date.1,due_in_date,invoice_currency,document_type,total_open_amount,cust_payment_terms,due_month,due_dayofweek,due_quarter
23850,CA02,0140104409,LOB,2019.0,9.500000e+09,2019-03-29,2018-10-30,2018-11-09,CAD,X2,383.64,CA10,11,4,4
14801,CA02,0140104409,LOB corp,2019.0,9.500000e+09,2019-03-29,2018-12-04,2018-12-14,CAD,X2,305.64,CA10,12,4,4
25463,U001,0200726979,BJ'S trust,2019.0,1.928538e+09,2018-12-30,2018-12-30,2019-01-14,USD,RV,241.59,NAA8,1,0,1
1593,U001,0200769623,WAL-MAR associates,2019.0,1.928542e+09,2018-12-30,2018-12-30,2019-01-14,USD,RV,38289.77,NAH4,1,0,1
11223,U001,0200769623,WAL-MAR corporation,2019.0,1.928543e+09,2018-12-30,2018-12-30,2019-01-14,USD,RV,16067.29,NAH4,1,0,1


In [36]:
#due month  val
X_val['due_month'] = X_val['due_in_date'].dt.month
#week
X_val['due_dayofweek'] = X_val['due_in_date'].dt.dayofweek
#quater
X_val['due_quarter'] = X_val['due_in_date'].dt.quarter


In [37]:
#due month due day of the week test
X_test['due_month'] = X_test['due_in_date'].dt.month
#week
X_test['due_dayofweek'] = X_test['due_in_date'].dt.dayofweek
#quater
X_test['due_quarter'] = X_test['due_in_date'].dt.quarter


<ipython-input-37-fe1de0236c3a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['due_month'] = X_test['due_in_date'].dt.month
<ipython-input-37-fe1de0236c3a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['due_dayofweek'] = X_test['due_in_date'].dt.dayofweek
<ipython-input-37-fe1de0236c3a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

In [38]:
X_test.head()

,business_code,cust_number,name_customer,buisness_year,doc_id,posting_date,document_create_date.1,due_in_date,invoice_currency,document_type,total_open_amount,cust_payment_terms,due_month,due_dayofweek,due_quarter
33418,U001,0200797452,US in,2019.0,1.930165e+09,2019-11-19,2019-11-19,2019-12-04,USD,RV,55487.28,NAA8,12,2,4
14820,U001,0200794332,COST systems,2019.0,1.930164e+09,2019-11-19,2019-11-19,2019-12-04,USD,RV,20591.60,NAAX,12,2,4
37703,U001,CCU013,KRAFT F llc,2019.0,1.930158e+09,2019-11-19,2019-11-19,2019-11-19,USD,RV,555.48,NAX2,11,1,4
30924,U001,0200704858,WAKE corp,2019.0,1.930166e+09,2019-11-19,2019-11-19,2019-12-04,USD,RV,132013.52,NAA8,12,2,4
29458,U001,0200744019,TARG corporation,2019.0,1.930157e+09,2019-11-19,2019-11-19,2019-12-04,USD,RV,80807.80,NAA8,12,2,4


In [39]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27315 entries, 23850 to 22199
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   business_code           27315 non-null  object        
 1   cust_number             27315 non-null  object        
 2   name_customer           27315 non-null  object        
 3   buisness_year           27315 non-null  float64       
 4   doc_id                  27315 non-null  float64       
 5   posting_date            27315 non-null  datetime64[ns]
 6   document_create_date.1  27315 non-null  datetime64[ns]
 7   due_in_date             27315 non-null  datetime64[ns]
 8   invoice_currency        27315 non-null  object        
 9   document_type           27315 non-null  object        
 10  total_open_amount       27315 non-null  float64       
 11  cust_payment_terms      27315 non-null  object        
 12  due_month               27315 non-null  in

In [40]:
#checking unique vals
X_train.business_code.value_counts()
#since there 6 vals applying label encoding

U001    24620
CA02     2222
U013      365
U002       96
U005        7
U007        5
Name: business_code, dtype: int64

In [41]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [42]:
X_train.document_type = le.fit_transform(X_train.document_type)
X_test.document_type = le.fit_transform(X_test.document_type)
X_val.document_type = le.fit_transform(X_val.document_type)

C:\Users\KIIT\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [43]:
# encoding business code
X_train.business_code = le.fit_transform(X_train.business_code)
X_val.business_code = le.fit_transform(X_val.business_code)
X_test.business_code = le.fit_transform(X_test.business_code)

In [44]:
X_train.head()

,business_code,cust_number,name_customer,buisness_year,doc_id,posting_date,document_create_date.1,due_in_date,invoice_currency,document_type,total_open_amount,cust_payment_terms,due_month,due_dayofweek,due_quarter
23850,0,0140104409,LOB,2019.0,9.500000e+09,2019-03-29,2018-10-30,2018-11-09,CAD,1,383.64,CA10,11,4,4
14801,0,0140104409,LOB corp,2019.0,9.500000e+09,2019-03-29,2018-12-04,2018-12-14,CAD,1,305.64,CA10,12,4,4
25463,1,0200726979,BJ'S trust,2019.0,1.928538e+09,2018-12-30,2018-12-30,2019-01-14,USD,0,241.59,NAA8,1,0,1
1593,1,0200769623,WAL-MAR associates,2019.0,1.928542e+09,2018-12-30,2018-12-30,2019-01-14,USD,0,38289.77,NAH4,1,0,1
11223,1,0200769623,WAL-MAR corporation,2019.0,1.928543e+09,2018-12-30,2018-12-30,2019-01-14,USD,0,16067.29,NAH4,1,0,1


In [45]:
#creating column which contains gap b/w document create date and due in date
X_train["doc_due_gap"] = X_train["due_in_date"] - X_train["document_create_date.1"]
X_train["doc_due_gap"] = X_train["doc_due_gap"].dt.days.astype('int32')
X_train.head()

,business_code,cust_number,name_customer,buisness_year,doc_id,posting_date,document_create_date.1,due_in_date,invoice_currency,document_type,total_open_amount,cust_payment_terms,due_month,due_dayofweek,due_quarter,doc_due_gap
23850,0,0140104409,LOB,2019.0,9.500000e+09,2019-03-29,2018-10-30,2018-11-09,CAD,1,383.64,CA10,11,4,4,10
14801,0,0140104409,LOB corp,2019.0,9.500000e+09,2019-03-29,2018-12-04,2018-12-14,CAD,1,305.64,CA10,12,4,4,10
25463,1,0200726979,BJ'S trust,2019.0,1.928538e+09,2018-12-30,2018-12-30,2019-01-14,USD,0,241.59,NAA8,1,0,1,15
1593,1,0200769623,WAL-MAR associates,2019.0,1.928542e+09,2018-12-30,2018-12-30,2019-01-14,USD,0,38289.77,NAH4,1,0,1,15
11223,1,0200769623,WAL-MAR corporation,2019.0,1.928543e+09,2018-12-30,2018-12-30,2019-01-14,USD,0,16067.29,NAH4,1,0,1,15


In [46]:
#doc due val
X_val["doc_due_gap"] = X_val["due_in_date"] - X_val["document_create_date.1"]
X_val["doc_due_gap"] = X_val["doc_due_gap"].dt.days.astype('int32')
X_val.head()

,business_code,cust_number,name_customer,buisness_year,doc_id,posting_date,document_create_date.1,due_in_date,invoice_currency,document_type,total_open_amount,cust_payment_terms,due_month,due_dayofweek,due_quarter,doc_due_gap
9440,1,CCU013,KRAFT F us,2019.0,1.929796e+09,2019-08-30,2019-08-30,2019-08-30,USD,0,10252.55,NAX2,8,4,3,0
40697,1,0200078854,MC A co,2019.0,1.929811e+09,2019-08-30,2019-08-30,2019-09-14,USD,0,2089.91,NAA8,9,5,3,15
32423,1,0200769623,WAL-MAR foundation,2019.0,1.929813e+09,2019-08-30,2019-08-30,2019-09-14,USD,0,1898.20,NAH4,9,5,3,15
30947,1,0200769623,WAL-MAR foundation,2019.0,1.929809e+09,2019-08-30,2019-08-30,2019-09-14,USD,0,200.47,NAH4,9,5,3,15
3960,1,0200794332,COST in,2019.0,1.929812e+09,2019-08-30,2019-08-30,2019-09-14,USD,0,87698.63,NAAX,9,5,3,15


In [47]:
#doc due test
X_test["doc_due_gap"] = X_test["due_in_date"] - X_test["document_create_date.1"]
X_test["doc_due_gap"] = X_test["doc_due_gap"].dt.days.astype('int32')
X_test.head()

<ipython-input-47-b7db300a3b1d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["doc_due_gap"] = X_test["due_in_date"] - X_test["document_create_date.1"]
<ipython-input-47-b7db300a3b1d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["doc_due_gap"] = X_test["doc_due_gap"].dt.days.astype('int32')


,business_code,cust_number,name_customer,buisness_year,doc_id,posting_date,document_create_date.1,due_in_date,invoice_currency,document_type,total_open_amount,cust_payment_terms,due_month,due_dayofweek,due_quarter,doc_due_gap
33418,1,0200797452,US in,2019.0,1.930165e+09,2019-11-19,2019-11-19,2019-12-04,USD,0,55487.28,NAA8,12,2,4,15
14820,1,0200794332,COST systems,2019.0,1.930164e+09,2019-11-19,2019-11-19,2019-12-04,USD,0,20591.60,NAAX,12,2,4,15
37703,1,CCU013,KRAFT F llc,2019.0,1.930158e+09,2019-11-19,2019-11-19,2019-11-19,USD,0,555.48,NAX2,11,1,4,0
30924,1,0200704858,WAKE corp,2019.0,1.930166e+09,2019-11-19,2019-11-19,2019-12-04,USD,0,132013.52,NAA8,12,2,4,15
29458,1,0200744019,TARG corporation,2019.0,1.930157e+09,2019-11-19,2019-11-19,2019-12-04,USD,0,80807.80,NAA8,12,2,4,15


In [48]:
#creating a column that contains quater of document create date 1
X_train['doc_cr_quarter'] = X_train['document_create_date.1'].dt.quarter
X_val['doc_cr_quarter'] = X_val['document_create_date.1'].dt.quarter
X_test['doc_cr_quarter'] = X_test['document_create_date.1'].dt.quarter

<ipython-input-48-f92f4e8d9950>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['doc_cr_quarter'] = X_test['document_create_date.1'].dt.quarter


In [49]:
X_test.head()

,business_code,cust_number,name_customer,buisness_year,doc_id,posting_date,document_create_date.1,due_in_date,invoice_currency,document_type,total_open_amount,cust_payment_terms,due_month,due_dayofweek,due_quarter,doc_due_gap,doc_cr_quarter
33418,1,0200797452,US in,2019.0,1.930165e+09,2019-11-19,2019-11-19,2019-12-04,USD,0,55487.28,NAA8,12,2,4,15,4
14820,1,0200794332,COST systems,2019.0,1.930164e+09,2019-11-19,2019-11-19,2019-12-04,USD,0,20591.60,NAAX,12,2,4,15,4
37703,1,CCU013,KRAFT F llc,2019.0,1.930158e+09,2019-11-19,2019-11-19,2019-11-19,USD,0,555.48,NAX2,11,1,4,0,4
30924,1,0200704858,WAKE corp,2019.0,1.930166e+09,2019-11-19,2019-11-19,2019-12-04,USD,0,132013.52,NAA8,12,2,4,15,4
29458,1,0200744019,TARG corporation,2019.0,1.930157e+09,2019-11-19,2019-11-19,2019-12-04,USD,0,80807.80,NAA8,12,2,4,15,4


In [50]:
#percentage  of customer no frequency
z = (X_train['cust_number'].value_counts() / X_train.cust_number.count())*100
z1 = z.to_dict() #converts to dictionary
X_train['Custno_freq'] = X_train['cust_number'].map(z1) 

In [51]:
#function to bin the values
def binningfun(col, cut_points, labels = None):
    minval = 0
    maxval = 100
    break_points = [minval] + cut_points +[maxval]
    if not labels:
        labels = range(len(cut_points)+1)
    colBin = pd.cut(col, bins=break_points, labels=labels, include_lowest=True)
    return colBin

In [52]:
cut_points = [5,10]
labels = ["l", "m", "h"]
X_train["cust_no_freq"] = binningfun(X_train["Custno_freq"], cut_points, labels)

map_cust_no_freq = {"l": 0, "m": 1, "h":2}

X_train['coded_frq'] = X_train['cust_no_freq'].map(map_cust_no_freq)

In [53]:

X_train

,business_code,cust_number,name_customer,buisness_year,doc_id,posting_date,document_create_date.1,due_in_date,invoice_currency,document_type,total_open_amount,cust_payment_terms,due_month,due_dayofweek,due_quarter,doc_due_gap,doc_cr_quarter,Custno_freq,cust_no_freq,coded_frq
23850,0,0140104409,LOB,2019.0,9.500000e+09,2019-03-29,2018-10-30,2018-11-09,CAD,1,383.64,CA10,11,4,4,10,4,0.955519,l,0
14801,0,0140104409,LOB corp,2019.0,9.500000e+09,2019-03-29,2018-12-04,2018-12-14,CAD,1,305.64,CA10,12,4,4,10,4,0.955519,l,0
25463,1,0200726979,BJ'S trust,2019.0,1.928538e+09,2018-12-30,2018-12-30,2019-01-14,USD,0,241.59,NAA8,1,0,1,15,4,3.624382,l,0
1593,1,0200769623,WAL-MAR associates,2019.0,1.928542e+09,2018-12-30,2018-12-30,2019-01-14,USD,0,38289.77,NAH4,1,0,1,15,4,21.160535,h,2
11223,1,0200769623,WAL-MAR corporation,2019.0,1.928543e+09,2018-12-30,2018-12-30,2019-01-14,USD,0,16067.29,NAH4,1,0,1,15,4,21.160535,h,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44109,1,0200769623,WAL-MAR us,2019.0,1.929802e+09,2019-08-30,2019-08-30,2019-09-14,USD,0,20800.00,NAH4,9,5,3,15,3,21.160535,h,2
39699,1,0200936373,SUPE corporation,2019.0,1.929817e+09,2019-08-30,2019-08-30,2019-09-14,USD,0,94051.47,NAA8,9,5,3,15,3,0.589420,l,0
22077,1,0200755701,ASSOCI co,2019.0,1.929806e+09,2019-08-30,2019-08-30,2019-09-14,USD,0,12729.92,NAA8,9,5,3,15,3,0.047593,l,0
31995,1,0200769623,WAL-MAR,2019.0,1.929811e+09,2019-08-30,2019-08-30,2019-09-14,USD,0,27714.31,NAH4,9,5,3,15,3,21.160535,h,2


In [54]:
X_train.drop(['Custno_freq'], axis=1,inplace=True)

In [55]:
#for validation set
X_val['Custno_freq'] = X_val['cust_number'].map(z1) 
X_val["cust_no_freq"] = binningfun(X_val["Custno_freq"], cut_points, labels)

X_val['cust_no_freq'].fillna(value='h', inplace=True)
X_val['coded_frq'] = X_val['cust_no_freq'].map(map_cust_no_freq)
X_val.drop(['Custno_freq'], axis=1,inplace=True)

X_val

,business_code,cust_number,name_customer,buisness_year,doc_id,posting_date,document_create_date.1,due_in_date,invoice_currency,document_type,total_open_amount,cust_payment_terms,due_month,due_dayofweek,due_quarter,doc_due_gap,doc_cr_quarter,cust_no_freq,coded_frq
9440,1,CCU013,KRAFT F us,2019.0,1.929796e+09,2019-08-30,2019-08-30,2019-08-30,USD,0,10252.55,NAX2,8,4,3,0,3,l,0
40697,1,0200078854,MC A co,2019.0,1.929811e+09,2019-08-30,2019-08-30,2019-09-14,USD,0,2089.91,NAA8,9,5,3,15,3,l,0
32423,1,0200769623,WAL-MAR foundation,2019.0,1.929813e+09,2019-08-30,2019-08-30,2019-09-14,USD,0,1898.20,NAH4,9,5,3,15,3,h,2
30947,1,0200769623,WAL-MAR foundation,2019.0,1.929809e+09,2019-08-30,2019-08-30,2019-09-14,USD,0,200.47,NAH4,9,5,3,15,3,h,2
3960,1,0200794332,COST in,2019.0,1.929812e+09,2019-08-30,2019-08-30,2019-09-14,USD,0,87698.63,NAAX,9,5,3,15,3,l,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44826,1,0200762301,C&S WH corporation,2019.0,1.930166e+09,2019-11-19,2019-11-19,2019-12-04,USD,0,19879.92,NAC6,12,2,4,15,4,l,0
931,1,0200769623,WAL-MAR corp,2019.0,1.930165e+09,2019-11-19,2019-11-19,2019-12-04,USD,0,14658.50,NAH4,12,2,4,15,4,h,2
3489,1,0200792293,UNIFIE corp,2019.0,1.930166e+09,2019-11-19,2019-11-19,2019-12-04,USD,0,51234.77,NAA8,12,2,4,15,4,l,0
9859,1,0200875006,KROGER us,2019.0,1.930169e+09,2019-11-19,2019-11-19,2019-12-04,USD,0,137970.49,NAA8,12,2,4,15,4,l,0


In [56]:

X_val.drop(['cust_no_freq'], axis=1,inplace=True)

In [57]:
#for test set
X_test['Custno_freq'] = X_test['cust_number'].map(z1) 
X_test["cust_no_freq"] = binningfun(X_test["Custno_freq"], cut_points, labels)
X_test['cust_no_freq'].fillna(value='h', inplace=True)

X_test['coded_frq'] = X_test['cust_no_freq'].map(map_cust_no_freq)
X_test.drop(['Custno_freq'], axis=1,inplace=True)

X_test

<ipython-input-57-7c92f3008f5d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['Custno_freq'] = X_test['cust_number'].map(z1)
<ipython-input-57-7c92f3008f5d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["cust_no_freq"] = binningfun(X_test["Custno_freq"], cut_points, labels)
C:\Users\KIIT\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.p

,business_code,cust_number,name_customer,buisness_year,doc_id,posting_date,document_create_date.1,due_in_date,invoice_currency,document_type,total_open_amount,cust_payment_terms,due_month,due_dayofweek,due_quarter,doc_due_gap,doc_cr_quarter,cust_no_freq,coded_frq
33418,1,0200797452,US in,2019.0,1.930165e+09,2019-11-19,2019-11-19,2019-12-04,USD,0,55487.28,NAA8,12,2,4,15,4,l,0
14820,1,0200794332,COST systems,2019.0,1.930164e+09,2019-11-19,2019-11-19,2019-12-04,USD,0,20591.60,NAAX,12,2,4,15,4,l,0
37703,1,CCU013,KRAFT F llc,2019.0,1.930158e+09,2019-11-19,2019-11-19,2019-11-19,USD,0,555.48,NAX2,11,1,4,0,4,l,0
30924,1,0200704858,WAKE corp,2019.0,1.930166e+09,2019-11-19,2019-11-19,2019-12-04,USD,0,132013.52,NAA8,12,2,4,15,4,l,0
29458,1,0200744019,TARG corporation,2019.0,1.930157e+09,2019-11-19,2019-11-19,2019-12-04,USD,0,80807.80,NAA8,12,2,4,15,4,l,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17887,1,0200515231,US,2020.0,1.930580e+09,2020-02-27,2020-02-27,2020-03-18,USD,0,27829.20,NAD1,3,2,1,20,1,l,0
38365,1,0200794332,COST corp,2020.0,1.930574e+09,2020-02-27,2020-02-27,2020-03-13,USD,0,8793.09,NAAX,3,4,1,15,1,l,0
28145,1,0200789077,US llc,2020.0,1.930567e+09,2020-02-27,2020-02-27,2020-03-13,USD,0,31094.52,NAA8,3,4,1,15,1,l,0
31641,1,0200413833,LDC systems,2020.0,1.930559e+09,2020-02-27,2020-02-27,2020-03-13,USD,0,29832.09,NAA8,3,4,1,15,1,l,0


In [58]:
X_test.cust_no_freq.value_counts()

l    6701
h    2405
m       0
Name: cust_no_freq, dtype: int64

In [59]:
print(X_test.cust_no_freq.isnull().sum())
X_test.drop(['cust_no_freq'], axis=1,inplace=True)

0


C:\Users\KIIT\anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [60]:
X_train = X_train.drop(columns=['doc_id','posting_date','cust_number', 'name_customer', 'document_create_date.1', 'due_in_date', 'invoice_currency' ])
X_train = X_train.drop(columns=['cust_payment_terms'])
X_train.drop(['cust_no_freq'], axis=1,inplace=True)
X_train.head()

,business_code,buisness_year,document_type,total_open_amount,due_month,due_dayofweek,due_quarter,doc_due_gap,doc_cr_quarter,coded_frq
23850,0,2019.0,1,383.64,11,4,4,10,4,0
14801,0,2019.0,1,305.64,12,4,4,10,4,0
25463,1,2019.0,0,241.59,1,0,1,15,4,0
1593,1,2019.0,0,38289.77,1,0,1,15,4,2
11223,1,2019.0,0,16067.29,1,0,1,15,4,2


In [61]:
X_val = X_val.drop(columns=['doc_id','posting_date','cust_number', 'name_customer', 'document_create_date.1', 'due_in_date', 'invoice_currency' ])
X_val = X_val.drop(columns=['cust_payment_terms'])
X_val.head()

,business_code,buisness_year,document_type,total_open_amount,due_month,due_dayofweek,due_quarter,doc_due_gap,doc_cr_quarter,coded_frq
9440,1,2019.0,0,10252.55,8,4,3,0,3,0
40697,1,2019.0,0,2089.91,9,5,3,15,3,0
32423,1,2019.0,0,1898.20,9,5,3,15,3,2
30947,1,2019.0,0,200.47,9,5,3,15,3,2
3960,1,2019.0,0,87698.63,9,5,3,15,3,0


In [62]:
X_test = X_test.drop(columns=['doc_id','posting_date','cust_number', 'name_customer', 'document_create_date.1', 'due_in_date', 'invoice_currency' ])
X_test = X_test.drop(columns=['cust_payment_terms'])
X_test.head()

,business_code,buisness_year,document_type,total_open_amount,due_month,due_dayofweek,due_quarter,doc_due_gap,doc_cr_quarter,coded_frq
33418,1,2019.0,0,55487.28,12,2,4,15,4,0
14820,1,2019.0,0,20591.60,12,2,4,15,4,0
37703,1,2019.0,0,555.48,11,1,4,0,4,0
30924,1,2019.0,0,132013.52,12,2,4,15,4,0
29458,1,2019.0,0,80807.80,12,2,4,15,4,0


In [63]:
#



In [64]:
#

In [65]:
#X_train = X_train.drop(columns=['coded_frq'])
#X_val = X_val.drop(columns=['coded_frq'])
#X_test = X_test.drop(columns=['coded_frq'])#\

In [66]:
#X_test['coded_frq'].fillna(value='0', inplace=True)

In [67]:
X_test.coded_frq.value_counts()

0    6701
2    2405
1       0
Name: coded_frq, dtype: int64

In [68]:
from sklearn.preprocessing import StandardScaler

In [69]:
X_train_cp = X_train.copy()
X_val_cp = X_val.copy()
X_test_cp = X_test.copy()

In [70]:
scaling = StandardScaler()


In [166]:
X_train_s = scaling.fit_transform(X_train)

In [72]:
X_val_s = scaling.transform(X_val)

In [73]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train_s,y_train)

LinearRegression()

In [74]:
accuracy = regressor.score(X_val_s,y_val)
print(accuracy*100,'%')

12.412741543932881 %


In [130]:
from sklearn.ensemble import RandomForestRegressor

In [131]:
RandomForestRegressor = RandomForestRegressor(n_estimators=50, random_state =0)
RandomForestRegressor.fit(X_train, y_train)

RandomForestRegressor(n_estimators=50, random_state=0)

In [132]:
accuracy = RandomForestRegressor.score(X_val,y_val)
print(accuracy*100,'%')

18.788734649804184 %


In [ ]:
from sklearn.linear_model import LogisticRegression
logModel = LogisticRegression()
param_grid = [    
    {'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
    'max_iter' : [100, 1000,2500, 5000]
    }
]

from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(logModel, param_grid = param_grid, cv = 3,verbose=True, n_jobs=-1)

best_clf = clf.fit(X_train_s,y_train)
best_clf.best_estimator_
print (f'Accuracy - : {best_clf.score(X_train_s,y_train):.3f}')